# 20251024卵巣癌HGSCと正常卵巣を比較できるRNA-seqデータセットを探す

- 「超鉄板レシピ」本p.224を参考にキュレーションする

## 1. NCBI GEOで検索
("high grade serous ovarian carcinoma"[All Fields] OR "HGSC"[All Fields]) AND "Expression profiling by high throughput sequencing"[Filter] AND "Homo sapiens"[Organism]
これで99データセットまで絞り込んだ

## 2. サマリーのリストをダウンロード->CSVに整形する
- .txt形式でダウンロードできた
- 1データセット1行で、サンプル数やプラットフォーム、コントロールの有無などの情報をまとめたい
- 全部手打ちは困難なので、まずはダウンロードした.txtファイルをCSVに整形する

## 3. 整形したCSVに項目を足して、有用なデータセットが一目でわかるようにする

## 4. 同じことがすぐできるように、スクリプトをgithubにプッシュしておく

## 2. サマリのリストをCSVに整形する

### セル1. 読み込み設定

In [ ]:
# parse_geo_series.py
# 必要なパッケージをインストール
import re, sys, csv, pathlib
import pandas as pd

# 読み込みファイルを指定
# パスを適宜変更してください  

def win_to_wsl(win_path: str) -> str:
    # r"C:\Users\foo\bar\baz.txt" → "/mnt/c/Users/foo/bar/baz.txt"
    return "/mnt/" + win_path[0].lower() + win_path[2:].replace("\\", "/")

win_path = r"C:\Users\nakan\Dropbox\02_大学院\02_S-HFD\20251024_卵巣癌RNAseqデータセットを探す\gds_result.txt"
path = pathlib.Path(win_to_wsl(win_path))
print("WSL path:", path, "exists:", path.exists())


# テキスト読み込み
txt = path.read_text(encoding="utf-8", errors="ignore")

# 読み込んだテキストの基本情報と先頭を表示
print("chars:", len(txt))
print("first 300 chars:\n", txt[:300])

WSL path: /mnt/c/Users/nakan/Dropbox/02_大学院/02_S-HFD/20251024_卵巣癌RNAseqデータセットを探す/gds_result.txt exists: True
chars: 82411
first 300 chars:
 
1. Multi-omics data analysis for the JGOG3025-TR2 ovarian cancer cohort
(Submitter supplied) This SuperSeries is composed of the SubSeries listed below.
Organism:	Homo sapiens
Type:		Expression profiling by high throughput sequencing; Methylation profiling by genome tiling array
Platforms: GPL24676


### セル2. データセットごとに分割する


In [5]:
# 「1.」「2.」などの行頭番号で分割 → 各データセットが blocks の1要素になる
blocks = re.split(r"\n(?=\d+\.\s)", txt.strip())

print("検出データセット数:", len(blocks))
print("---- 最初のブロックの冒頭 ----")
print(blocks[0][:600])

検出データセット数: 99
---- 最初のブロックの冒頭 ----
1. Multi-omics data analysis for the JGOG3025-TR2 ovarian cancer cohort
(Submitter supplied) This SuperSeries is composed of the SubSeries listed below.
Organism:	Homo sapiens
Type:		Expression profiling by high throughput sequencing; Methylation profiling by genome tiling array
Platforms: GPL24676 GPL21145 GPL21558 863 Samples
FTP download: GEO (CEL, IDAT, OSCHP, TXT) ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE263nnn/GSE263455/
Series		Accession: GSE263455	ID: 200263455



### セル3. 情報抽出と表(dataframe)作成

In [17]:
rows = []
for b in blocks:
    block = re.sub(r"\r", "", b)

    def grab(pat):
        m = re.search(pat, block, re.IGNORECASE | re.DOTALL)
        return m.group(1).strip() if m else ""

    title = grab(r"^\d+\.\s*(.+?)\n")
    organism = grab(r"Organism:\s*([^\n]+)")
    type_ = grab(r"Type:\s*([^\n\r]+)")
    platform_raw = grab(r"Platforms?:\s*([^\n\r]+)")
    gpls = re.findall(r"GPL\d+", platform_raw)
    platform = " ".join(gpls) if gpls else platform_raw.split(" Samples")[0].strip()
    gse = grab(r"Accession:\s*(GSE\d+)")
    samples = grab(r"(\d+)\s+Samples?")
    has_ctrl = "Yes" if re.search(r"(normal|control|fallopian|tube)", block, re.I) else ""

    rows.append({
        "GSE_ID": gse,
        "Title": title,
        "Organism": organism,
        "Type": type_,
        "Platform": platform,
        "Samples": samples,
        "Has_control_hint": has_ctrl
    })

df = pd.DataFrame(rows)
df.head(3)

,GSE_ID,Title,Organism,Type,Platform,Samples,Has_control_hint
0,GSE263455,Multi-omics data analysis for the JGOG3025-TR2...,Homo sapiens,Expression profiling by high throughput sequen...,GPL24676 GPL21145 GPL21558,863,
1,GSE263083,Multi-omics data analysis for the JGOG3025-TR2...,Homo sapiens,Expression profiling by high throughput sequen...,GPL24676,291,
2,GSE102094,Molecular analysis of high-grade serous ovaria...,Homo sapiens,Expression profiling by high throughput sequen...,GPL6801 GPL16791 GPL17904,286,


### セル5. 手入力用のカラムを追加

In [18]:

# 追加したい列（手入力用）
new_cols = [
    "publication_year",      # 例: 2021
    "PMID",                  # 例: 33312345
    "tissue",                # 例: ovary / fallopian tube / omentum など
    "rna_type",              # 例: polyA+ / rRNA-depleted / total RNA
    "library_layout",        # 例: PE / SE
    "vivo_or_vitro",         # 例: in vivo / in vitro
    "paired_control_available",  # 例: yes / no / unknown
    "comments"               # 備考
]

# まだ無い列だけ追加（空文字で初期化）
for c in new_cols:
    if c not in df.columns:
        df[c] = pd.Series([""] * len(df), dtype="string")

# 列の並びを整える（既存 → 追加の順）
preferred_front = [
    "GSE_ID", "Title", "Organism", "Type", "Platform", "Samples", "Has_control_hint"
]
ordered = [c for c in preferred_front if c in df.columns] + \
          [c for c in new_cols if c in df.columns] + \
          [c for c in df.columns if c not in preferred_front + new_cols]
df = df.reindex(columns=ordered)

df.head(3)

,GSE_ID,Title,Organism,Type,Platform,Samples,Has_control_hint,publication_year,PMID,tissue,rna_type,library_layout,vivo_or_vitro,paired_control_available,comments
0,GSE263455,Multi-omics data analysis for the JGOG3025-TR2...,Homo sapiens,Expression profiling by high throughput sequen...,GPL24676 GPL21145 GPL21558,863,,,,,,,,,
1,GSE263083,Multi-omics data analysis for the JGOG3025-TR2...,Homo sapiens,Expression profiling by high throughput sequen...,GPL24676,291,,,,,,,,,
2,GSE102094,Molecular analysis of high-grade serous ovaria...,Homo sapiens,Expression profiling by high throughput sequen...,GPL6801 GPL16791 GPL17904,286,,,,,,,,,


In [19]:

# 保存（Excelでも文字化けしないUTF-8 BOM付き）
out_csv = path.with_name("geo_series_summary_for_selection.csv")
df.to_csv(out_csv, index=False, encoding="utf-8-sig")
print("✅ 追加入力用CSVを保存:", out_csv)


✅ 追加入力用CSVを保存: /mnt/c/Users/nakan/Dropbox/02_大学院/02_S-HFD/20251024_卵巣癌RNAseqデータセットを探す/geo_series_summary_for_selection.csv
